Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [5]:
batch_size = 128
def train_1(batch_size, num_steps):
    graph = tf.Graph()
    with graph.as_default():

      with tf.name_scope('input'):
        tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
        tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
        tf_valid_dataset = tf.constant(valid_dataset)
        tf_test_dataset = tf.constant(test_dataset)
        beta_regul = tf.placeholder(tf.float32)

      with tf.name_scope('lay_1'): 
        weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]), name='weights')
        biases = tf.Variable(tf.zeros([num_labels]), name='biases')

      logits = tf.matmul(tf_train_dataset, weights) + biases

      with tf.name_scope('loss'):
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + beta_regul * tf.nn.l2_loss(weights)

      with tf.name_scope('train'):
        train_step = tf.train.GradientDescentOptimizer(0.2).minimize(loss)

      # Predictions for the training, validation, and test data.
      train_prediction = tf.nn.softmax(logits)
      valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
      test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

    with tf.Session(graph=graph) as session:
      tf.global_variables_initializer().run()
      print("Initialized")
      for step in range(num_steps):    
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)    
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]   
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, beta_regul : 1e-3}
        _, l, predictions = session.run(
          [train_step, loss, train_prediction], feed_dict=feed_dict)
        if (step % 1000 == 0):
          print("Minibatch loss at step %d: %f" % (step, l))
          print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
          print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
      print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

    print("Regularization Test:")
    regul_val = [pow(10, i) for i in np.arange(-4, -2, 0.1)]
    for regul in regul_val:
      with tf.Session(graph=graph) as session:
        tf.global_variables_initializer().run()
        for step in range(num_steps):    
          offset = (step * batch_size) % (train_labels.shape[0] - batch_size)  
          batch_data = train_dataset[offset:(offset + batch_size), :]
          batch_labels = train_labels[offset:(offset + batch_size), :] 
          feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, beta_regul : regul}
          _, l, predictions = session.run([train_step, loss, train_prediction], feed_dict=feed_dict)
        acc = accuracy(test_prediction.eval(), test_labels)   
        print("Regularization: %f;  Accuracy: %f" % (regul, acc))   
    
%time train_1(128, 3001)

Initialized
Minibatch loss at step 0: 19.383762
Minibatch accuracy: 7.8%
Validation accuracy: 12.3%
Minibatch loss at step 1000: 3.192081
Minibatch accuracy: 75.0%
Validation accuracy: 76.2%
Minibatch loss at step 2000: 1.807954
Minibatch accuracy: 85.2%
Validation accuracy: 77.9%
Minibatch loss at step 3000: 1.512782
Minibatch accuracy: 78.1%
Validation accuracy: 79.5%
Test accuracy: 87.0%
Regularization Test:
Regularization: 0.000100;  Accuracy: 84.850000
Regularization: 0.000126;  Accuracy: 84.760000
Regularization: 0.000158;  Accuracy: 85.030000
Regularization: 0.000200;  Accuracy: 85.460000
Regularization: 0.000251;  Accuracy: 85.220000
Regularization: 0.000316;  Accuracy: 85.410000
Regularization: 0.000398;  Accuracy: 85.530000
Regularization: 0.000501;  Accuracy: 85.900000
Regularization: 0.000631;  Accuracy: 85.900000
Regularization: 0.000794;  Accuracy: 86.660000
Regularization: 0.001000;  Accuracy: 86.970000
Regularization: 0.001259;  Accuracy: 87.530000
Regularization: 0.001

the 1-layer neural network:
L2 loss for a tensor t using nn.l2_loss(t). 

In [6]:
def train_1_layer(batch_size, num_hidden_nodes, num_steps):
    graph = tf.Graph()
    with graph.as_default():
        with tf.name_scope('input'): 
            tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
            tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
            tf_valid_dataset = tf.constant(valid_dataset)
            tf_test_dataset = tf.constant(test_dataset)
            beta_regul = tf.placeholder(tf.float32)

        with tf.name_scope('lay_1'): 
            weights1 = tf.Variable(tf.truncated_normal([image_size * image_size, num_hidden_nodes]))
            biases1 = tf.Variable(tf.zeros([num_hidden_nodes]))
            weights2 = tf.Variable(tf.truncated_normal([num_hidden_nodes, num_labels]))
            biases2 = tf.Variable(tf.zeros([num_labels]))

        # Training
        lay1_train = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)
        logits = tf.matmul(lay1_train, weights2) + biases2

        with tf.name_scope('loss'):
            loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + \
            beta_regul * (tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights2))

        with tf.name_scope('train'):
            train_step = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

        # Predictions for the training, validation, and test data.
        train_prediction = tf.nn.softmax(logits)
        lay1_valid = tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1)
        valid_prediction = tf.nn.softmax(tf.matmul(lay1_valid, weights2) + biases2)
        lay1_test = tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1)
        test_prediction = tf.nn.softmax(tf.matmul(lay1_test, weights2) + biases2)
    
    with tf.Session(graph=graph) as session:
        tf.global_variables_initializer().run()
        print("Initialized")
        for step in range(num_steps):           
            offset = (step * batch_size) % (train_labels.shape[0] - batch_size)           
            batch_data = train_dataset[offset:(offset + batch_size), :]
            batch_labels = train_labels[offset:(offset + batch_size), :]           
            feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, beta_regul : 1e-3}
            _, l, predictions = session.run([train_step, loss, train_prediction], feed_dict=feed_dict)
            if (step % 1000 == 0):
                print("Step: %d" % step)
                print("  Minibatch loss: %f" % l)
                print("  Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
                print("  Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
        print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))
    
    print("Regularization Test:")
    regul_val = [pow(10, i) for i in np.arange(-4, -2, 0.1)]   
    for regul in regul_val:    
      with tf.Session(graph=graph) as session:
        tf.global_variables_initializer().run()
        for step in range(num_steps):         
          offset = (step * batch_size) % (train_labels.shape[0] - batch_size)        
          batch_data = train_dataset[offset:(offset + batch_size), :]
          batch_labels = train_labels[offset:(offset + batch_size), :]         
          feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, beta_regul : regul}
          _, l, predictions = session.run([train_step, loss, train_prediction], feed_dict=feed_dict)
        acc = accuracy(test_prediction.eval(), test_labels)       
        print("Regularization: %f;  Accuracy: %f" % (regul, acc))  
        
%time train_1_layer(128, 1024, 3001)    

Initialized
Step: 0
  Minibatch loss: 650.942749
  Minibatch accuracy: 9.4%
  Validation accuracy: 24.6%
Step: 1000
  Minibatch loss: 114.686707
  Minibatch accuracy: 80.5%
  Validation accuracy: 82.0%
Step: 2000
  Minibatch loss: 41.256401
  Minibatch accuracy: 86.7%
  Validation accuracy: 84.6%
Step: 3000
  Minibatch loss: 15.465127
  Minibatch accuracy: 85.9%
  Validation accuracy: 86.6%
Test accuracy: 92.8%
Regularization Test:
Regularization: 0.000100;  Accuracy: 89.830000
Regularization: 0.000126;  Accuracy: 89.400000
Regularization: 0.000158;  Accuracy: 90.270000
Regularization: 0.000200;  Accuracy: 89.870000
Regularization: 0.000251;  Accuracy: 90.350000
Regularization: 0.000316;  Accuracy: 90.720000
Regularization: 0.000398;  Accuracy: 91.380000
Regularization: 0.000501;  Accuracy: 91.450000
Regularization: 0.000631;  Accuracy: 92.050000
Regularization: 0.000794;  Accuracy: 92.480000
Regularization: 0.001000;  Accuracy: 92.820000
Regularization: 0.001259;  Accuracy: 93.060000


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [7]:
def train_overfiting_model(batch_size, num_hidden_nodes, num_steps, num_bacthes) :     
    graph = tf.Graph()
    with graph.as_default():
        with tf.name_scope('input'):
            tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
            tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
            tf_valid_dataset = tf.constant(valid_dataset)
            tf_test_dataset = tf.constant(test_dataset)
            beta_regul = tf.placeholder(tf.float32)

        with tf.name_scope('lay_1'): 
            weights1 = tf.Variable(tf.truncated_normal([image_size * image_size, num_hidden_nodes]))
            biases1 = tf.Variable(tf.zeros([num_hidden_nodes]))
            weights2 = tf.Variable(tf.truncated_normal([num_hidden_nodes, num_labels]))
            biases2 = tf.Variable(tf.zeros([num_labels]))

        # Training
        lay1_train = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)
        logits = tf.matmul(lay1_train, weights2) + biases2

        with tf.name_scope('loss'):
            loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
            tf.summary.scalar('loss', loss)

        with tf.name_scope('train'):
            train_step = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

        # Predictions for the training, validation, and test data.
        train_prediction = tf.nn.softmax(logits)
        lay1_valid = tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1)
        valid_prediction = tf.nn.softmax(tf.matmul(lay1_valid, weights2) + biases2)
        lay1_test = tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1)
        test_prediction = tf.nn.softmax(tf.matmul(lay1_test, weights2) + biases2)

    with tf.Session(graph=graph) as session:
        tf.global_variables_initializer().run()
        print("Initialized")
        for step in range(num_steps):           
            offset = step % num_bacthes           
            batch_data = train_dataset[offset:(offset + batch_size), :]
            batch_labels = train_labels[offset:(offset + batch_size), :]           
            feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, beta_regul : 1e-3}
            _, l, predictions = session.run([train_step, loss, train_prediction], feed_dict=feed_dict)
            if (step % 20 == 0):
                print("Step: %d" % step)
                print("  Minibatch loss: %f" % l)
                print("  Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
                print("  Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
        print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))  
    
%time train_overfiting_model(128, 1024, 101, 3)

Initialized
Step: 0
  Minibatch loss: 319.790009
  Minibatch accuracy: 8.6%
  Validation accuracy: 35.5%
Step: 20
  Minibatch loss: 0.000000
  Minibatch accuracy: 100.0%
  Validation accuracy: 60.1%
Step: 40
  Minibatch loss: 0.000000
  Minibatch accuracy: 100.0%
  Validation accuracy: 60.1%
Step: 60
  Minibatch loss: 0.000000
  Minibatch accuracy: 100.0%
  Validation accuracy: 60.1%
Step: 80
  Minibatch loss: 0.000000
  Minibatch accuracy: 100.0%
  Validation accuracy: 60.1%
Step: 100
  Minibatch loss: 0.000000
  Minibatch accuracy: 100.0%
  Validation accuracy: 60.1%
Test accuracy: 66.4%
Wall time: 7.02 s


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [8]:
def train_drop_model(batch_size, num_hidden_nodes, num_steps, num_batches):  
    graph = tf.Graph()
    with graph.as_default():
        with tf.name_scope('input'): 
            tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
            tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
            tf_valid_dataset = tf.constant(valid_dataset)
            tf_test_dataset = tf.constant(test_dataset)

        with tf.name_scope('lay_1'): 
            weights1 = tf.Variable(tf.truncated_normal([image_size * image_size, num_hidden_nodes]))
            biases1 = tf.Variable(tf.zeros([num_hidden_nodes]))
            weights2 = tf.Variable(tf.truncated_normal([num_hidden_nodes, num_labels]))
            biases2 = tf.Variable(tf.zeros([num_labels]))

        # Training computation.
        lay1_train = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)
        drop1 = tf.nn.dropout(lay1_train, 0.5)
        logits = tf.matmul(drop1, weights2) + biases2
        
        with tf.name_scope('loss'):        
            loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))

        with tf.name_scope('train'):
            train_step = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

        # Predictions for the training, validation, and test data.
        train_prediction = tf.nn.softmax(logits)
        lay1_valid = tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1)
        valid_prediction = tf.nn.softmax(tf.matmul(lay1_valid, weights2) + biases2)
        lay1_test = tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1)
        test_prediction = tf.nn.softmax(tf.matmul(lay1_test, weights2) + biases2)

    with tf.Session(graph=graph) as session:
        tf.global_variables_initializer().run()
        print("Initialized")
        for step in range(num_steps):           
            offset = step % num_batches           
            batch_data = train_dataset[offset:(offset + batch_size), :]
            batch_labels = train_labels[offset:(offset + batch_size), :]           
            feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
            _, l, predictions = session.run(
              [train_step, loss, train_prediction], feed_dict=feed_dict)
            if (step % 20 == 0):
                print("Step: %d" % step)
                print("  Minibatch loss: %f" % l)
                print("  Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
                print("  Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
        print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

%time train_drop_model(128, 1024, 101, 3)

Initialized
Step: 0
  Minibatch loss: 499.434204
  Minibatch accuracy: 8.6%
  Validation accuracy: 28.9%
Step: 20
  Minibatch loss: 11.294602
  Minibatch accuracy: 96.1%
  Validation accuracy: 67.2%
Step: 40
  Minibatch loss: 5.894017
  Minibatch accuracy: 97.7%
  Validation accuracy: 69.3%
Step: 60
  Minibatch loss: 0.000000
  Minibatch accuracy: 100.0%
  Validation accuracy: 67.8%
Step: 80
  Minibatch loss: 1.095686
  Minibatch accuracy: 99.2%
  Validation accuracy: 68.1%
Step: 100
  Minibatch loss: 0.000000
  Minibatch accuracy: 100.0%
  Validation accuracy: 68.2%
Test accuracy: 75.8%
Wall time: 7.35 s


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [9]:
def train_2_hidden_layers_model(batch_size, num_hidden_nodes1, num_hidden_nodes2, beta_regul, num_steps): 
    graph = tf.Graph()
    with graph.as_default():
        with tf.name_scope('input'): 
            tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
            tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
            tf_valid_dataset = tf.constant(valid_dataset)
            tf_test_dataset = tf.constant(test_dataset)
            global_step = tf.Variable(0)

        with tf.name_scope('layers'):
            with tf.name_scope('weights1'):
                weights1 = tf.Variable(tf.truncated_normal([image_size * image_size, num_hidden_nodes1], stddev=np.sqrt(2.0 / (image_size * image_size))))
            with tf.name_scope('biases1'): 
                biases1 = tf.Variable(tf.zeros([num_hidden_nodes1]))
            with tf.name_scope('weights2'):
                weights2 = tf.Variable(tf.truncated_normal([num_hidden_nodes1, num_hidden_nodes2], stddev=np.sqrt(2.0 / num_hidden_nodes1)))
            with tf.name_scope('biases2'):                 
                biases2 = tf.Variable(tf.zeros([num_hidden_nodes2]))
            with tf.name_scope('weights3'):                
                weights3 = tf.Variable(tf.truncated_normal([num_hidden_nodes2, num_labels], stddev=np.sqrt(2.0 / num_hidden_nodes2)))
            with tf.name_scope('biases2'): 
                biases3 = tf.Variable(tf.zeros([num_labels]))

        # Training computation.
        lay1_train = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)
        lay2_train = tf.nn.relu(tf.matmul(lay1_train, weights2) + biases2)
        logits = tf.matmul(lay2_train, weights3) + biases3
        with tf.name_scope('loss'):
            loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + \
              beta_regul * (tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights2) + tf.nn.l2_loss(weights3))
            tf.summary.scalar('loss', loss)

        with tf.name_scope('train'):
            learning_rate = tf.train.exponential_decay(0.5, global_step, 1000, 0.65, staircase=True)
            train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

        # Predictions for the training, validation, and test data.
        train_prediction = tf.nn.softmax(logits)
        lay1_valid = tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1)
        lay2_valid = tf.nn.relu(tf.matmul(lay1_valid, weights2) + biases2)
        valid_prediction = tf.nn.softmax(tf.matmul(lay2_valid, weights3) + biases3)
        lay1_test = tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1)
        lay2_test = tf.nn.relu(tf.matmul(lay1_test, weights2) + biases2)
        test_prediction = tf.nn.softmax(tf.matmul(lay2_test, weights3) + biases3)

    with tf.Session(graph=graph) as session:
        tf.global_variables_initializer().run()
        print("Initialized")
        for step in range(num_steps):           
            offset = (step * batch_size) % (train_labels.shape[0] - batch_size)           
            batch_data = train_dataset[offset:(offset + batch_size), :]
            batch_labels = train_labels[offset:(offset + batch_size), :]         
            feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
            _, l, predictions = session.run(
              [train_step, loss, train_prediction], feed_dict=feed_dict)
            if (step % 1000 == 0):
                print("Step: %d" % step)
                print("  Minibatch loss: %f" % l)
                print("  Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
                print("  Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
        print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))
        
%time train_2_hidden_layers_model(128, 1024, 100, 0.003, 10001)

Initialized
Step: 0
  Minibatch loss: 5.007732
  Minibatch accuracy: 8.6%
  Validation accuracy: 32.4%
Step: 1000
  Minibatch loss: 0.794802
  Minibatch accuracy: 83.6%
  Validation accuracy: 85.5%
Step: 2000
  Minibatch loss: 0.474745
  Minibatch accuracy: 92.2%
  Validation accuracy: 86.7%
Step: 3000
  Minibatch loss: 0.560658
  Minibatch accuracy: 87.5%
  Validation accuracy: 87.1%
Step: 4000
  Minibatch loss: 0.471579
  Minibatch accuracy: 90.6%
  Validation accuracy: 87.9%
Step: 5000
  Minibatch loss: 0.534287
  Minibatch accuracy: 89.1%
  Validation accuracy: 88.2%
Step: 6000
  Minibatch loss: 0.617279
  Minibatch accuracy: 85.2%
  Validation accuracy: 88.2%
Step: 7000
  Minibatch loss: 0.569403
  Minibatch accuracy: 87.5%
  Validation accuracy: 88.5%
Step: 8000
  Minibatch loss: 0.684139
  Minibatch accuracy: 83.6%
  Validation accuracy: 88.6%
Step: 9000
  Minibatch loss: 0.552442
  Minibatch accuracy: 89.1%
  Validation accuracy: 88.7%
Step: 10000
  Minibatch loss: 0.506989
  M

In [10]:
def train_3_hidden_layers_model(batch_size, num_hidden_nodes1, num_hidden_nodes2, num_hidden_nodes3, keep_prob, num_steps, decay_steps, decay_rate): 
    graph = tf.Graph()
    with graph.as_default():
        with tf.name_scope('input'):    
            tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
            tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
            tf_valid_dataset = tf.constant(valid_dataset)
            tf_test_dataset = tf.constant(test_dataset)
            global_step = tf.Variable(0)
        with tf.name_scope('layer'):
            with tf.name_scope('weights1'):
                weights1 = tf.Variable(tf.truncated_normal([image_size * image_size, num_hidden_nodes1], stddev=np.sqrt(2.0 / (image_size * image_size))))
            with tf.name_scope('biases1'):                
                biases1 = tf.Variable(tf.zeros([num_hidden_nodes1]))
            with tf.name_scope('weights2'):    
                weights2 = tf.Variable(tf.truncated_normal([num_hidden_nodes1, num_hidden_nodes2], stddev=np.sqrt(2.0 / num_hidden_nodes1)))
            with tf.name_scope('biases2'):  
                biases2 = tf.Variable(tf.zeros([num_hidden_nodes2]))
            with tf.name_scope('weights3'):            
                weights3 = tf.Variable(tf.truncated_normal([num_hidden_nodes2, num_hidden_nodes3], stddev=np.sqrt(2.0 / num_hidden_nodes2)))
            with tf.name_scope('biases3'):  
                biases3 = tf.Variable(tf.zeros([num_hidden_nodes3]))
            with tf.name_scope('weights4'):            
                weights4 = tf.Variable(tf.truncated_normal([num_hidden_nodes3, num_labels], stddev=np.sqrt(2.0 / num_hidden_nodes3)))
            with tf.name_scope('biases4'):                 
                biases4 = tf.Variable(tf.zeros([num_labels]))
          
            lay1_train = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)
            lay2_train = tf.nn.relu(tf.matmul(lay1_train, weights2) + biases2)
            lay3_train = tf.nn.relu(tf.matmul(lay2_train, weights3) + biases3)
            
        logits = tf.matmul(lay3_train, weights4) + biases4
        with tf.name_scope('loss'):
            loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
            tf.summary.scalar('loss', loss)

        with tf.name_scope('train'):
            learning_rate = tf.train.exponential_decay(0.5, global_step, decay_steps, decay_rate, staircase=True)
            optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

        # Predictions for the training, validation, and test data.
        train_prediction = tf.nn.softmax(logits)
        lay1_valid = tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1)
        lay2_valid = tf.nn.relu(tf.matmul(lay1_valid, weights2) + biases2)
        lay3_valid = tf.nn.relu(tf.matmul(lay2_valid, weights3) + biases3)
        valid_prediction = tf.nn.softmax(tf.matmul(lay3_valid, weights4) + biases4)
        
        lay1_test = tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1)
        lay2_test = tf.nn.relu(tf.matmul(lay1_test, weights2) + biases2)
        lay3_test = tf.nn.relu(tf.matmul(lay2_test, weights3) + biases3)
        test_prediction = tf.nn.softmax(tf.matmul(lay3_test, weights4) + biases4)
    
    with tf.Session(graph=graph) as session:
        tf.global_variables_initializer().run()
        print("Initialized")
        for step in range(num_steps):           
            offset = (step * batch_size) % (train_labels.shape[0] - batch_size)           
            batch_data = train_dataset[offset:(offset + batch_size), :]
            batch_labels = train_labels[offset:(offset + batch_size), :]          
            feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
            _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
            if (step % 2000 == 0):
                print("Step: %d" % step)
                print("  Minibatch loss: %f" % l)
                print("  Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
                print("  Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
        print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))       

In [11]:
%time train_3_hidden_layers_model(128, 1024, 256, 128, 0.5, 20001, 4000, 0.6) 

Initialized
Step: 0
  Minibatch loss: 2.317394
  Minibatch accuracy: 13.3%
  Validation accuracy: 33.3%
Step: 2000
  Minibatch loss: 0.256093
  Minibatch accuracy: 93.8%
  Validation accuracy: 88.5%
Step: 4000
  Minibatch loss: 0.262931
  Minibatch accuracy: 93.0%
  Validation accuracy: 89.5%
Step: 6000
  Minibatch loss: 0.382675
  Minibatch accuracy: 92.2%
  Validation accuracy: 90.1%
Step: 8000
  Minibatch loss: 0.246397
  Minibatch accuracy: 93.0%
  Validation accuracy: 90.1%
Step: 10000
  Minibatch loss: 0.149402
  Minibatch accuracy: 95.3%
  Validation accuracy: 90.4%
Step: 12000
  Minibatch loss: 0.130671
  Minibatch accuracy: 96.9%
  Validation accuracy: 90.8%
Step: 14000
  Minibatch loss: 0.087093
  Minibatch accuracy: 98.4%
  Validation accuracy: 90.6%
Step: 16000
  Minibatch loss: 0.032714
  Minibatch accuracy: 99.2%
  Validation accuracy: 90.9%
Step: 18000
  Minibatch loss: 0.033141
  Minibatch accuracy: 99.2%
  Validation accuracy: 91.0%
Step: 20000
  Minibatch loss: 0.0574

In [12]:
%time train_3_hidden_layers_model(128, 1024, 256, 128, 0.5, 20001, 4000, 0.7) 

Initialized
Step: 0
  Minibatch loss: 2.458052
  Minibatch accuracy: 6.2%
  Validation accuracy: 27.6%
Step: 2000
  Minibatch loss: 0.257144
  Minibatch accuracy: 93.8%
  Validation accuracy: 88.5%
Step: 4000
  Minibatch loss: 0.251111
  Minibatch accuracy: 92.2%
  Validation accuracy: 89.1%
Step: 6000
  Minibatch loss: 0.370334
  Minibatch accuracy: 88.3%
  Validation accuracy: 89.5%
Step: 8000
  Minibatch loss: 0.225524
  Minibatch accuracy: 92.2%
  Validation accuracy: 90.0%
Step: 10000
  Minibatch loss: 0.156632
  Minibatch accuracy: 94.5%
  Validation accuracy: 89.9%
Step: 12000
  Minibatch loss: 0.149752
  Minibatch accuracy: 96.1%
  Validation accuracy: 90.3%
Step: 14000
  Minibatch loss: 0.111866
  Minibatch accuracy: 96.9%
  Validation accuracy: 90.5%
Step: 16000
  Minibatch loss: 0.059365
  Minibatch accuracy: 98.4%
  Validation accuracy: 90.4%
Step: 18000
  Minibatch loss: 0.047696
  Minibatch accuracy: 98.4%
  Validation accuracy: 90.5%
Step: 20000
  Minibatch loss: 0.05794

In [13]:
def train_4_hidden_layers_model(batch_size, num_hidden_nodes1, num_hidden_nodes2, num_hidden_nodes3, num_hidden_nodes4, keep_prob, num_steps, decay_steps, decay_rate): 
    graph = tf.Graph()
    with graph.as_default():
        with tf.name_scope('input'): 
            tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
            tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
            tf_valid_dataset = tf.constant(valid_dataset)
            tf_test_dataset = tf.constant(test_dataset)
            global_step = tf.Variable(0)

        with tf.name_scope('4_layers'):
            weights1 = tf.Variable(tf.truncated_normal([image_size * image_size, num_hidden_nodes1], stddev=np.sqrt(2.0 / (image_size * image_size))))
            biases1 = tf.Variable(tf.zeros([num_hidden_nodes1]))
            weights2 = tf.Variable(tf.truncated_normal([num_hidden_nodes1, num_hidden_nodes2], stddev=np.sqrt(2.0 / num_hidden_nodes1)))
            biases2 = tf.Variable(tf.zeros([num_hidden_nodes2]))
            weights3 = tf.Variable(tf.truncated_normal([num_hidden_nodes2, num_hidden_nodes3], stddev=np.sqrt(2.0 / num_hidden_nodes2)))
            biases3 = tf.Variable(tf.zeros([num_hidden_nodes3]))
            weights4 = tf.Variable(tf.truncated_normal([num_hidden_nodes3, num_hidden_nodes4], stddev=np.sqrt(2.0 / num_hidden_nodes3)))
            biases4 = tf.Variable(tf.zeros([num_hidden_nodes4]))
            weights5 = tf.Variable(tf.truncated_normal([num_hidden_nodes4, num_labels], stddev=np.sqrt(2.0 / num_hidden_nodes4)))
            biases5 = tf.Variable(tf.zeros([num_labels]))
    
            # Training computation.
            lay1_train = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)
            drop1 = tf.nn.dropout(lay1_train, 0.5)
            lay2_train = tf.nn.relu(tf.matmul(drop1, weights2) + biases2)
            drop2 = tf.nn.dropout(lay2_train, 0.5)
            lay3_train = tf.nn.relu(tf.matmul(drop2, weights3) + biases3)
            drop3 = tf.nn.dropout(lay3_train, 0.5)        
            lay4_train = tf.nn.relu(tf.matmul(drop3, weights4) + biases4)
            drop4 = tf.nn.dropout(lay4_train, 0.5)
        
        logits = tf.matmul(drop4, weights5) + biases5

        with tf.name_scope('loss'):
            loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
            tf.summary.scalar('loss', loss)
            
        with tf.name_scope('train'):
            learning_rate = tf.train.exponential_decay(0.5, global_step, decay_steps, decay_rate, staircase=True)
            optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
        # Predictions for the training, validation, and test data.
        train_prediction = tf.nn.softmax(logits)
        lay1_valid = tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1)
        lay2_valid = tf.nn.relu(tf.matmul(lay1_valid, weights2) + biases2)
        lay3_valid = tf.nn.relu(tf.matmul(lay2_valid, weights3) + biases3)
        lay4_valid = tf.nn.relu(tf.matmul(lay3_valid, weights4) + biases4)
        valid_prediction = tf.nn.softmax(tf.matmul(lay4_valid, weights5) + biases5)
        lay1_test = tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1)
        lay2_test = tf.nn.relu(tf.matmul(lay1_test, weights2) + biases2)
        lay3_test = tf.nn.relu(tf.matmul(lay2_test, weights3) + biases3)
        lay4_test = tf.nn.relu(tf.matmul(lay3_test, weights4) + biases4)
        test_prediction = tf.nn.softmax(tf.matmul(lay4_test, weights5) + biases5)
     
    with tf.Session(graph=graph) as session:        
        tf.global_variables_initializer().run()
        print("Initialized")
        for step in range(num_steps):            
            offset = (step * batch_size) % (train_labels.shape[0] - batch_size)           
            batch_data = train_dataset[offset:(offset + batch_size), :]
            batch_labels = train_labels[offset:(offset + batch_size), :]           
            feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
            _, l, predictions = session.run(
              [optimizer, loss, train_prediction], feed_dict=feed_dict)
            if (step % 2000 == 0):
                print("Step: %d" % step)
                print("  Minibatch loss: %f" % l)
                print("  Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
                print("  Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
        print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))  

In [14]:
%time train_4_hidden_layers_model(128, 1024, 512, 256, 256, 0.5, 20001, 4000, 0.65) 

Initialized
Step: 0
  Minibatch loss: 3.016583
  Minibatch accuracy: 10.2%
  Validation accuracy: 18.1%
Step: 2000
  Minibatch loss: 0.472662
  Minibatch accuracy: 87.5%
  Validation accuracy: 85.5%
Step: 4000
  Minibatch loss: 0.399398
  Minibatch accuracy: 87.5%
  Validation accuracy: 86.9%
Step: 6000
  Minibatch loss: 0.581667
  Minibatch accuracy: 85.2%
  Validation accuracy: 87.7%
Step: 8000
  Minibatch loss: 0.691913
  Minibatch accuracy: 78.9%
  Validation accuracy: 88.3%
Step: 10000
  Minibatch loss: 0.486305
  Minibatch accuracy: 85.9%
  Validation accuracy: 88.6%
Step: 12000
  Minibatch loss: 0.483506
  Minibatch accuracy: 86.7%
  Validation accuracy: 88.9%
Step: 14000
  Minibatch loss: 0.402433
  Minibatch accuracy: 87.5%
  Validation accuracy: 89.4%
Step: 16000
  Minibatch loss: 0.381107
  Minibatch accuracy: 91.4%
  Validation accuracy: 89.3%
Step: 18000
  Minibatch loss: 0.293322
  Minibatch accuracy: 91.4%
  Validation accuracy: 89.5%
Step: 20000
  Minibatch loss: 0.4731

In [15]:
%time train_4_hidden_layers_model(128, 1024, 512, 256, 256, 0.5, 20001, 5000, 0.80) 

Initialized
Step: 0
  Minibatch loss: 2.908870
  Minibatch accuracy: 12.5%
  Validation accuracy: 12.5%
Step: 2000
  Minibatch loss: 0.420541
  Minibatch accuracy: 89.1%
  Validation accuracy: 85.7%
Step: 4000
  Minibatch loss: 0.531792
  Minibatch accuracy: 86.7%
  Validation accuracy: 86.8%
Step: 6000
  Minibatch loss: 0.529683
  Minibatch accuracy: 85.2%
  Validation accuracy: 87.5%
Step: 8000
  Minibatch loss: 0.710805
  Minibatch accuracy: 78.1%
  Validation accuracy: 88.0%
Step: 10000
  Minibatch loss: 0.421076
  Minibatch accuracy: 88.3%
  Validation accuracy: 88.2%
Step: 12000
  Minibatch loss: 0.439087
  Minibatch accuracy: 85.2%
  Validation accuracy: 88.4%
Step: 14000
  Minibatch loss: 0.457686
  Minibatch accuracy: 85.2%
  Validation accuracy: 89.0%
Step: 16000
  Minibatch loss: 0.331902
  Minibatch accuracy: 91.4%
  Validation accuracy: 89.2%
Step: 18000
  Minibatch loss: 0.283972
  Minibatch accuracy: 93.8%
  Validation accuracy: 89.2%
Step: 20000
  Minibatch loss: 0.4381